# Kerr spacetime in quasi-isotropic coordinates

## Check of the 3+1 Einstein equations

This SageMath notebook accompanies the article _High precision numerical sequences of rotating hairy black holes_ by
Gustavo Garcı́a, Eric Gourgoulhon, Philippe Grandclément and Marcelo Salgado ([arXiv:2302.06659](https://arxiv.org/abs/2302.06659)). It involves SageMath tools developed through the [SageManifolds](https://sagemanifolds.obspm.fr/) project. 

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [ ]:
%display latex

Since some computations are quite long, we ask for running them in parallel on 8 cores:

In [ ]:
Parallelism().set(nproc=8)

## Spacelike hypersurface $\Sigma$

We consider some hypersurface $\Sigma$ of a spacelike foliation $(\Sigma_t)_{t\in\mathbb{R}}$ of Kerr spacetime; we declare $\Sigma_t$ as a 3-dimensional Riemannian manifold:

In [ ]:
Sig = Manifold(3, 'Sigma', latex_name=r'\Sigma', 
               structure='Riemannian', metric_name='gam', 
               metric_latex_name=r'\gamma', start_index=1)
print(Sig)

On $\Sigma$, we consider the _"rational-polynomial" quasi-isotropic coordinates_ $(r,y,\phi)$ inheritated from the **quasi-isotropic (QI) coordinates** $(t,r,\theta,\phi)$ of Kerr spacetime, via $y=\cos\theta$:

In [ ]:
X.<r,y,ph> = Sig.chart(r'r:(1,+oo) y:(-1,1) ph:(0,2*pi):\phi')
print(X) ; X

<h3>Riemannian metric on $\Sigma$</h3>

<p>First the two Kerr parameters:</p>

In [ ]:
var('m, a', domain='real')
assume(m > 0)
assume(a > 0)
assumptions()

A shortcut for $\cos(2\theta)$:

In [ ]:
cos2t = 2*y^2 - 1 

The metric coefficients $A^2$ and $B^2$:

In [ ]:
A2 = (-2*a^2*m*(m + 4*r) + 8*a^2*r^2*cos2t + a^4 + (m + 2*r)^4) / (16*r^4)
A2

In [ ]:
B2n = (8*a^2*r^2*cos2t*(a^2 - m^2 + 4*r^2)^2 - 4*a^6*(m^2 +4*m*r - 2*r^2)
       + a^4*(96*m^2*r^2 + 48*m^3*r + 6*m^4 - 64*m*r^3 + 32*r^4)
       - 4*a^2*(m + 2*r)^2*(18*m^2*r^2 + 8*m^3*r + m^4 - 8*m*r^3 - 8*r^4)
       + a^8 + (m + 2*r)^8)

B2d = 16*r^4*(-2*a^2*m*(m + 4*r) + 8*a^2*r^2*cos2t + a^4 + (m + 2*r)^4)

B2 = B2n / B2d
B2

The metric $\gamma$ induced by the spacetime metric $g$ on $\Sigma$:

In [ ]:
gam = Sig.metric()
gam[1,1] = A2
gam[2,2] = A2*r^2/(1 - y^2)
gam[3,3] = B2*r^2*(1 - y^2)
gam.display()

A matrix view of the components w.r.t. coordinates $(r,y,\phi)$:

In [ ]:
gam[:]

<h3>Lapse function and shift vector</h3>

In [ ]:
N1 = (1 - sqrt(m^2 - a^2)/(2*r))*(1 + sqrt(m^2 - a^2)/(2*r)) / sqrt(B2)
N1

In [ ]:
N = Sig.scalar_field(N1, name='N')
print(N)
N.display()

In [ ]:
b = Sig.vector_field(name='beta', latex_name=r'\beta') 
b[3] = -2*a*m/(r^3*(A2*B2))*(1 + m/r + (m^2 - a^2)/(4*r^2))  
# unset components are zero 
b.display()

### Extrinsic curvature of $\Sigma$

We use the formula
$$ K_{ij} = \frac{1}{2N} \mathcal{L}_{\beta} \gamma_{ij} $$
which is valid for any stationary spacetime:

In [ ]:
K = gam.lie_derivative(b) / (2*N)
K.set_name('K')
print(K)

In [ ]:
K.display_comp(only_nonredundant=True)

<p>The type-(1,1) tensor $K^\sharp$ of components $K^i_{\ \, j} = \gamma^{ik} K_{kj}$:</p>

In [ ]:
Ku = K.up(gam, 0)
print(Ku)

<p>We may check that the hypersurface $\Sigma$ is maximal, i.e. that $K^k_{\ \, k} = 0$:</p>

In [ ]:
trK = Ku.trace()
print(trK)
trK.display()

<h3>Connection and curvature</h3>
<p>Let us call $D$ the Levi-Civita connection associated with $\gamma$: </p>

In [ ]:
D = gam.connection(name='D')
print(D)

<p>The Ricci tensor associated with $\gamma$:</p>

In [ ]:
Ric = gam.ricci()
print(Ric)

<p>The scalar curvature $R = \gamma^{ij} R_{ij}$:</p>

In [ ]:
R = gam.ricci_scalar(name='R')
print(R)

The coordinate expression of the scalar curvature is quite long:

In [ ]:
print(R.expr())

<h2>3+1 Einstein equations</h2>
<p>Let us check that the vacuum 3+1 Einstein equations are satisfied.</p>
<p>We start by the contraint equations:</p>
<h3>Hamiltonian constraint</h3>

<p>Let us first evaluate the term $K_{ij} K^{ij}$:</p>

In [ ]:
Kuu = Ku.up(gam, 1)
trKK = K['_ij']*Kuu['^ij']
print(trKK)

The vacuum Hamiltonian constraint equation is 
$$ R + K^2 -K_{ij} K^{ij} = 0 $$

In [ ]:
Ham = R + trK^2 - trKK
print(Ham)
Ham.display()

Hence the Hamiltonian constraint is fulfilled. 

### Momentum constraint

In vaccum, the momentum constraint is
$$ D_j K^j_{\ \, i} - D_i K = 0 $$

In [ ]:
mom = D(Ku).trace(0,2) - D(trK)
print(mom)
mom.display()

Hence the momentum constraint is fulfilled. 

<h3>Dynamical Einstein equations</h3>
<p>Let us first evaluate the symmetric bilinear form $k_{ij} := K_{ik} K^k_{\ \, j}$:</p>

In [ ]:
KK = K['_ik']*Ku['^k_j']
print(KK)

In [ ]:
KK1 = KK.symmetrize()
KK == KK1

In [ ]:
KK = KK1
print(KK)

<p>In vacuum and for stationary spacetimes, the dynamical Einstein equations are
$$ \mathcal{L}_\beta K_{ij} - D_i D_j N + N \left( R_{ij} + K K_{ij} - 2 K_{ik} K^k_{\ \, j}\right) = 0 $$

In [ ]:
dyn = K.lie_derivative(b) - D(D(N)) + N*( Ric + trK*K - 2*KK )
print(dyn)
dyn.display()

Hence the dynamical Einstein equations are fulfilled.